In [1]:
library("DESeq2")
library("EnhancedVolcano")
library(edgeR)
library(stringr)
library(gdata)
library(edgeR)
library(msigdbr)
library(hypeR)

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: 'BiocGenerics'


The following objects are masked from 'package:stats':

    IQR, mad, sd, var, xtabs


The following objects are masked from 'package:base':

    Filter, Find, Map, Position, Reduce, anyDuplicated, aperm, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, intersect, is.unsorted,
    lapply, mapply, match, mget, order, paste, pmax, pmax.int, pmin,
    pmin.int, rank, rbind, rownames, sapply, setdiff, sort, table,
    tapply, union, unique, unsplit, which.max, which.min



Attaching package: 'S4Vectors'


The following object is masked from 'package:utils':

    findMatches


The following objects are masked from 'package:base':

    I, expand.grid, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loa

In [6]:
getwd()

[1] "/data1/projects/2021_MOPA/SMALL_NMF_V2/github/tutorial"

In [10]:
source('a.R')

In [11]:
setwd('/data1/projects/2021_MOPA/SMALL_NMF_V2/github/')

# DEG

In [27]:
full_data <-read.table('data/gene_matrix.csv',sep=',',header=1)
sample_label<- read.table('data/sample_label.csv',sep=',',header=1)
####original data DESEQ2
col_data<-sample_label
rownames(col_data) <-col_data$X
rownames(full_data)<-full_data[,1]
full_data<-full_data[,-c(1)]
colnames(col_data)<-c('sample','Subtype')

In [28]:
rownames(full_data)<-full_data[,1]

TCGA.D5.6928,TCGA.CK.6746,TCGA.CK.5916,TCGA.D5.6931,TCGA.D5.6930,TCGA.AZ.4615,TCGA.CK.5913,TCGA.AA.3492,TCGA.CK.4951,TCGA.D5.6927,...,TCGA.CM.5349,TCGA.CM.5348,TCGA.CM.5344,TCGA.CM.4748,TCGA.CK.6748,TCGA.CK.4948,TCGA.CA.6719,TCGA.CA.6717,TCGA.D5.6929,TCGA.CM.5341
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1116,3062,3285,5040,2170,2349,2991,2356,6513,2031,...,9419,1244,8887,374,14359,9710,12566,2155,9839,9167
8,6,37,40,1,1,67,4,8,2,...,24,4,136,0,113,270,55,11,51,23
1630,1532,1115,1461,1144,960,1933,317,959,1427,...,3138,1788,2839,55,3820,3649,2670,1875,2346,3228
992,871,420,796,358,303,757,301,307,485,...,576,1131,382,20,746,968,611,650,560,550
328,457,323,757,320,173,540,396,246,287,...,501,350,297,10,259,814,514,325,328,287
2306,198,523,305,199,190,442,115,324,739,...,947,1595,252,4,573,807,476,742,305,300
5045,859,1079,1375,863,861,893,383,842,1248,...,1047,2091,1039,10,1470,1418,1513,2328,1053,1225
2617,2723,5328,4061,4160,4340,5224,3172,2604,3731,...,4209,3635,4660,167,4147,5533,5360,4113,4174,2759
1954,1364,2118,2329,1492,1467,2568,1139,1548,1628,...,1888,2261,1881,67,2485,5061,3878,3645,3644,709


In [29]:
len <-length(unique(col_data$Subtype))
#len<-len*(len-1)/2
empty_list <- vector(mode = "list", length = (len*(len-1))/2)
unique(sample_label$Subtype)[1]
num<-1
for( i in c(1:(length(unique(sample_label$Subtype))-1)))
{    
    for(j in c((i+1):length(unique(sample_label$Subtype))))
        {
            empty_list[[num]]<-append(empty_list[[num]],unique(sample_label$Subtype)[i])
            empty_list[[num]]<-append(empty_list[[num]],unique(sample_label$Subtype)[j])
            num<-num+1
    }
 }
len<-len*(len-1)/2
colnames(full_data)<-col_data[,1]


for( i in c(1:len))
{
    dds<-list()
    rownames(col_data)<-col_data[,1]
    a<- DESeqDataSetFromMatrix(countData =full_data, 
                               colData = col_data[col_data$Subtype %in% empty_list[[i]],],
                               design = ~ Subtype)
    print(i)
    dds <- estimateSizeFactors(a)
    dds<- DESeq(dds)
    res<-list()
    res[[1]]<-results(dds)
    res<-data.frame(res[[1]])
    res$rownames<-rownames(res)
    res<-as.data.frame(res)
    print(4)
    res<-list()
    res[[1]]<-results(dds)
    
    res[[1]]$symbol <-full_data[,1]
    #res[[1]]$ensemble <-featuredata$ensemble
    #res[[1]]
    keyvals.colour <- ifelse(res[[1]]$log2FoldChange < -2&res[[1]]$padj<0.05, 'royalblue',
                             ifelse(res[[1]]$log2FoldChange > 2&res[[1]]$padj<0.05, 'red',
                                    ifelse(res[[1]]$log2FoldChange< 2 & res[[1]]$log2FoldChange> -2& res[[1]]$padj> 0.05,'darkgreen','grey')))
    names(keyvals.colour)[keyvals.colour == 'red'] <- 'high'
    names(keyvals.colour)[keyvals.colour == 'darkgreen'] <- 'mid'
    names(keyvals.colour)[keyvals.colour == 'royalblue'] <- 'low'
    EnhancedVolcano(toptable = res[[1]], # data-frame of test statistics
                    lab = res[[1]]$symbol, # column name in toptable containing variable names
                    x ='log2FoldChange', # A column name in toptable containing log2 fold change
                    y = 'padj', # A column name in toptable containing p-value
                    pCutoff = 0.05, # Cut-off for statistical significance. (in raw value)
                    FCcutoff = 2,   # Cut-off for absolute log2 fold-change
                    title = method, # Title of the plot
                    subtitle = "Fold Change > 2, padj < 0.05", # Subtitle of the plot
                    ylim = c(0, 15), # Limits of the y-axis
                    pCutoffCol = 'padj',
                    gridlines.minor = F,
                    legendPosition = 'none',
                    selectLab = rownames(res[[1]])[which(names(keyvals.colour) %in% c('high', 'low'))],
                    colCustom = keyvals.colour)
}

[1] "CMS1"

ERROR: Error in DESeqDataSetFromMatrix(countData = full_data, colData = col_data[col_data$Subtype %in% : ncol(countData) == nrow(colData) is not TRUE


In [35]:
    a<- DESeqDataSetFromMatrix(countData =full_data[,colnames(full_data)%in%rownames(col_data[col_data$Subtype %in% empty_list[[i]],])], 
                               colData = col_data[col_data$Subtype %in% empty_list[[i]],],
                               design = ~ Subtype)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
"some variables in design formula are characters, converting to factors"


In [37]:
print(i)
    dds <- estimateSizeFactors(a)
    dds<- DESeq(dds)
    res<-list()
    res[[1]]<-results(dds)
    res<-data.frame(res[[1]])
    res$rownames<-rownames(res)
    res<-as.data.frame(res)
    print(4)
    res<-list()
    res[[1]]<-results(dds)

[1] 1


using pre-existing size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 2105 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] 4


In [39]:
res[[1]]

log2 fold change (MLE): Subtype CMS2 vs CMS1 
Wald test p-value: Subtype CMS2 vs CMS1 
DataFrame with 56602 rows and 6 columns
       baseMean log2FoldChange     lfcSE       stat      pvalue        padj
      <numeric>      <numeric> <numeric>  <numeric>   <numeric>   <numeric>
1     7701.7502       1.780443 0.1391651   12.79375 1.77693e-37 1.30601e-34
2       60.1966       3.428591 0.3783782    9.06128 1.28927e-19 8.69511e-18
3     2855.3776       1.210974 0.1080514   11.20739 3.75089e-29 8.77172e-27
4      670.6469       0.265548 0.0795635    3.33756 8.45161e-04 3.44268e-03
5      498.1626       0.382516 0.1050992    3.63957 2.73094e-04 1.27152e-03
...         ...            ...       ...        ...         ...         ...
56598  0.146817      0.1015080  1.095952  0.0926208   0.9262048          NA
56599  0.174818     -0.0177697  1.006863 -0.0176486   0.9859192          NA
56600  0.110085     -0.2072352  1.169328 -0.1772260   0.8593309          NA
56601 30.899980      0.3077139  0.176

In [ ]:
res[[1]]$symbol <-full_data[,1]
#res[[1]]$ensemble <-featuredata$ensemble
#res[[1]]
keyvals.colour <- ifelse(res[[1]]$log2FoldChange < -2&res[[1]]$padj<0.05, 'royalblue',
                         ifelse(res[[1]]$log2FoldChange > 2&res[[1]]$padj<0.05, 'red',
                                ifelse(res[[1]]$log2FoldChange< 2 & res[[1]]$log2FoldChange> -2& res[[1]]$padj> 0.05,'darkgreen','grey')))
names(keyvals.colour)[keyvals.colour == 'red'] <- 'high'
names(keyvals.colour)[keyvals.colour == 'darkgreen'] <- 'mid'
names(keyvals.colour)[keyvals.colour == 'royalblue'] <- 'low'
EnhancedVolcano(toptable = res[[1]], # data-frame of test statistics
                lab = res[[1]]$symbol, # column name in toptable containing variable names
                x ='log2FoldChange', # A column name in toptable containing log2 fold change
                y = 'padj', # A column name in toptable containing p-value
                pCutoff = 0.05, # Cut-off for statistical significance. (in raw value)
                FCcutoff = 2,   # Cut-off for absolute log2 fold-change
                title = method, # Title of the plot
                subtitle = "Fold Change > 2, padj < 0.05", # Subtitle of the plot
                ylim = c(0, 15), # Limits of the y-axis
                pCutoffCol = 'padj',
                gridlines.minor = F,
                legendPosition = 'none',
                selectLab = rownames(res[[1]])[which(names(keyvals.colour) %in% c('high', 'low'))],
                colCustom = keyvals.colour)

In [2]:
KEGG= msigdbr(species = "human", category = "C2",subcategory = 'CP:KEGG')
REACTOME = msigdbr(species = "human", category = "C2",subcategory = 'CP:REACTOME')
GO = msigdbr(species = "human", category = "C5")
reactome_pathway = split(x = REACTOME$gene_symbol, f = REACTOME$gs_name)
kegg_pathway = split(x = KEGG$gene_symbol, f = KEGG$gs_name)

In [3]:
reactome_pathway = split(x = REACTOME$ensembl_gene, f = REACTOME$gs_name)
kegg_pathway = split(x = KEGG$ensembl_gene, f = KEGG$gs_name)
GO_pathway = split(x = GO$ensembl_gene, f = GO$gs_name)
hyp_obj<- hypeR(DEG, reactome_pathway , test="hypergeometric")
hyp_obj<-hyp_obj$data[hyp_obj$data$fdr<0.05,]

In [ ]:
##

In [ ]:
GGPLOT2